In [10]:
import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSplitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel


In [11]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats',  'athlete_info'])
data = loader.load()

/Users/ryanli/Documents/WOD-prediction/wod_predictor/data_loader.py:56: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
/Users/ryanli/Documents/WOD-prediction/wod_predictor/data_loader.py:56: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [12]:
splitter = DataSplitter(sample = 20000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [13]:
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_method': 'standard', # standard or quantile
        'conversion_method': 'rpm'

    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero',
        'drop_missing_threshold': 0.9
        # 'neighbors': 5,
    },
    'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)


NameError: name 'conversion_method' is not defined

In [ ]:
train_preprocessed = preprocessor.transform(data=train_data)

In [ ]:
test_preprocessed = preprocessor.transform(data=test_data)

In [ ]:
config = {}
rf_modeler = RandomForestModel(config = config, n_estimators = 10)
rf_modeler.fit(X = train_preprocessed['X'], y = train_preprocessed['y'])
rf_modeler.predict(X = test_preprocessed['X'])
rf_modeler.show_results(y_test = test_preprocessed['y'], meta_data = test_preprocessed['meta_data'])